In [17]:
# Install dependencies with pip (needed for Codespaces)
%pip install numpy
%pip install qiskit
%pip install qiskit_ibm_runtime
%pip install matplotlib

import sys
import math
import base64
import random
import hashlib
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Operator
from numpy import pi

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [18]:
import sys
import random

# Increase the recursion limit for deep recursive functions
sys.setrecursionlimit(1500)

def gcd(a, b):
    """Compute the greatest common divisor of a and b."""
    while b:
        a, b = b, a % b
    return a

def modular_pow(base, exponent, modulus):
    """Perform modular exponentiation."""
    result = 1
    base = base % modulus
    while exponent > 0:
        if (exponent % 2) == 1:  # If exponent is odd, multiply base with the result
            result = (result * base) % modulus
        exponent = exponent >> 1  # exponent = exponent / 2
        base = (base * base) % modulus  # base = base * base
    return result

def fermat(p, k=2):
    """Test if p is a prime number using Fermat's little theorem, repeated k times."""
    if p < 2:
        return False
    if p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]:
        return True
    if any(p % i == 0 for i in [2, 3, 5]):
        return False
    for _ in range(k):
        a = random.randint(2, p - 2)
        if modular_pow(a, p - 1, p) != 1:
            return False
    return True

def karatsuba(x, y):
    """Multiply x and y using the Karatsuba multiplication algorithm."""
    if x < 10 or y < 10:
        return x * y
    m = max(len(str(x)), len(str(y))) // 2
    high1, low1 = divmod(x, 10**m)
    high2, low2 = divmod(y, 10**m)
    z0 = karatsuba(low1, low2)
    z1 = karatsuba((low1 + high1), (low2 + high2))
    z2 = karatsuba(high1, high2)
    return z2 * 10**(2*m) + ((z1 - z2 - z0) * 10**m) + z0

def mod_inverse(a, m):
    """Compute the modular inverse of a under modulo m using Extended Euclidean Algorithm."""
    m0, x0, x1 = m, 0, 1
    if m == 1:
        return 0
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1

def generate_e(phi):
    """Find an appropriate e and compute d using mod inverse."""
    for e in [65537, 32767, 17011, 8191, 4093, 2039, 1021, 509, 257, 127, 61, 37, 17]:
        if gcd(e, phi) == 1:
            d = mod_inverse(e, phi)
            return {'e': e, 'd': d}
    return None

def generate_rsa_keys():
    """Generate RSA public and private keys."""
    primes = []
    while len(primes) < 2:
        p = random.randint(2**15, 2**16)
        if fermat(p):
            primes.append(p)
    p, q = primes
    n = karatsuba(p, q)
    phi = karatsuba(p-1, q-1)
    e_d = generate_e(phi)
    return {'p': p, 'q': q, 'n': n, 'phi': phi, 'e': e_d['e'], 'd': e_d['d']}

keys = generate_rsa_keys()
print("Generated RSA keys:", keys)


Generated RSA keys: {'p': 52369, 'q': 41999, 'n': 2199445631, 'phi': 2199351264, 'e': 65537, 'd': 1077543521}


In [19]:
# Qiskit Shor's Algorithm Program
%pip install pylatexenc

# Necessary imports
import os
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from math import pi, log, ceil, sqrt
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler

# Classical preprocessing to check if quantum computation is necessary
def classical_preprocessing(N):
    if N % 2 == 0:
        return f"{N} is even, no need for quantum algorithm."
    for base in range(2, int(sqrt(N)) + 1):
        power = round(log(N, base))
        if base ** power == N:
            return f"{N} is a power of {base}^({power}), no need for quantum algorithm."
    return None

# Controlled unitary function for the modular multiplication
def c_amod15(a, power, n):
    total_qubits = n + 1
    U = QuantumCircuit(total_qubits)
    for _ in range(power):
        for q in range(n):
            U.swap(q, (q + 1) % n)
        U.x(range(n))
    return U.to_gate().control(1)

# Inverse Quantum Fourier Transform (QFT†)
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for j in range(n):
        for k in range(j):
            qc.cp(-pi/float(2**(j-k)), k, j)
        qc.h(j)
    return qc

# Quantum order finding circuit
def order_finding_circuit(N, a):
    n = ceil(log(N, 2))
    total_qubits = 2 * n + 1
    qc = QuantumCircuit(total_qubits, n)
    qc.h(range(n))
    for q in range(n):
        qc.append(c_amod15(a, 2**q, n), [q] + list(range(n, 2*n)) + [2*n])
    qc.append(qft_dagger(n), range(n))
    qc.measure(range(n), range(n))
    return qc

# Load or save IBM Quantum account credentials
IBM_TOKEN = os.getenv('IBM_QUANTUM_TOKEN')
if IBM_TOKEN:
    QiskitRuntimeService.save_account(
        channel="ibm_quantum",
        token=IBM_TOKEN,
        instance="ibm-q/open/main",
        set_as_default=True,
        overwrite=True
    )
else:
    print("IBM Quantum Token not set in environment variables.")

# Load the default service
service = QiskitRuntimeService()
backend = service.backend("ibmq_qasm_simulator")

# Generate RSA keys
keys = generate_rsa_keys()
print("Generated RSA keys:", keys)

# Set the number to be factored and the base for the order finding
N = keys['n']
a = 7  # You can choose a different base if needed

# Perform classical preprocessing and decide whether to proceed with quantum steps
preprocessing_result = classical_preprocessing(N)
if preprocessing_result:
    print(preprocessing_result)
else:
    qc = order_finding_circuit(N, a)
    # Transpile the circuit for the specific backend
    transpiled_circuit = transpile(qc, backend)
    
    # Use Sampler to run the transpiled circuit
    sampler = Sampler(backend)
    job = sampler.run(transpiled_circuit)
    print(f"job id: {job.job_id()}")
    result = job.result()
    print("")
    print("RESULTS:", result)
    print("PLOT:")
    quasi_dists = result.quasi_dists[0]  # Taking the first result if multiple

    # Create a bar plot
    plt.bar(quasi_dists.keys(), quasi_dists.values())
    plt.xlabel('Measured State')
    plt.ylabel('Probability')
    plt.title('Measurement Outcomes from Shor\'s Algorithm')
    plt.xticks(rotation=70)
    plt.show()

Note: you may need to restart the kernel to use updated packages.
Generated RSA keys: {'p': 59833, 'q': 54833, 'n': 3280822889, 'phi': 3280708224, 'e': 65537, 'd': 2713691393}
